In [15]:
from PIL import Image
import os
import glob
from keras.preprocessing import image
import keras.preprocessing
import tensorflow as tf

In [16]:
train_gen = image.ImageDataGenerator(
    rescale = 1/255.0,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_gen = image.ImageDataGenerator(rescale=1/255.0)

In [17]:
train_generator = train_gen.flow_from_directory(
    r"D:\\Github\\Oral-cancer\\Phase 1\\Tongue\\Train\\",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 139 images belonging to 2 classes.


In [18]:
train_generator.class_indices

{'Cancer': 0, 'Non Cancer': 1}

In [19]:
val_generator = test_gen.flow_from_directory(
    r"D:\\Github\\Oral-cancer\\Phase 1\\Tongue\\Val\\",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 8 images belonging to 2 classes.


In [20]:
# Create the VGG16 models

from keras.applications.vgg16 import VGG16
import keras
from keras.layers import *
from keras.models import Model 
import matplotlib.pyplot as plt

In [30]:
model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in model.layers:
    layer.trainable=False

In [31]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(64,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(1,activation='softmax')(d1)
model_new = Model(model.input,fc2)
model_new.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [32]:
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model_new.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])

In [33]:
hist = model_new.fit(
    train_generator,
    steps_per_epoch = 4,
    epochs = 10,
    validation_data = val_generator,
    validation_steps = 2,
)

Epoch 1/10
4/4 [==============================] - 6s 1s/step - loss: 0.7661 - accuracy: 0.4486 - val_loss: 0.6029 - val_accuracy: 0.6250
Epoch 2/10
4/4 [==============================] - 3s 609ms/step - loss: 0.7777 - accuracy: 0.4579
Epoch 3/10
4/4 [==============================] - 3s 614ms/step - loss: 0.6312 - accuracy: 0.4673
Epoch 4/10
4/4 [==============================] - 3s 831ms/step - loss: 0.7008 - accuracy: 0.4673
Epoch 5/10
4/4 [==============================] - 3s 619ms/step - loss: 0.5833 - accuracy: 0.4673
Epoch 6/10
4/4 [==============================] - 3s 640ms/step - loss: 0.5627 - accuracy: 0.4299
Epoch 7/10
4/4 [==============================] - 3s 619ms/step - loss: 0.5253 - accuracy: 0.4673
Epoch 8/10
4/4 [==============================] - 3s 876ms/step - loss: 0.4826 - accuracy: 0.4486
Epoch 9/10
4/4 [==============================] - 3s 605ms/step - loss: 0.5063 - accuracy: 0.5047
Epoch 10/10
4/4 [==============================] - 3s 611ms/step - loss: 0.4902